In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA

import sys
sys.path.append('..')
from gspneuro import data_loading as dload 
from gspneuro import plotting as viz
from gspneuro.brain import Brain

In [ ]:
# paths and loading subjects IDs
data_base_dir = "/Users/hugofluhr/chuv/data/"

healthy_ids = dload.get_subjects(data_base_dir)
atrophy_ids = dload.get_atrophies_subjects()

In [ ]:
atrophy_ids

In [ ]:
atrophy_info = dload.load_patient_infos()
id2diag={patient.get("id"):patient.get("diagnostic") for patient in atrophy_info}

In [ ]:
id2diag

In [ ]:
# Parameters
scale = 1
consensus_type = 'consensus_bin'


In [ ]:
healthy_thickness = [dload.get_signal(data_base_dir, healthy_subj, 'thickness', scale=scale) for healthy_subj in healthy_ids]
atrophy_thickness = [dload.get_atrophy_signal(atrophy_subj, scale=scale) for atrophy_subj in atrophy_ids]

In [ ]:
len(healthy_ids)

In [ ]:
consensus = Brain(data_base_dir, consensus_type, scale)
consensus.load_graph()

In [ ]:
avg_healthy_thick=np.stack(healthy_thickness).mean(axis=0)

# GFT of Thickness

In [ ]:
gft_controls = np.stack([consensus.G.gft(s) for s in healthy_thickness])
gft_atrophies = np.stack([consensus.G.gft(s) for s in atrophy_thickness])

avg_gft = gft_controls.mean(axis=0)

In [ ]:
# X = np.stack(gft_atrophies)
# pca = PCA(n_components=2)
# reduced_data = pca.fit_transform(X)

# for i, sub in enumerate(atrophy_info):
#     plt.scatter(reduced_data[i, 0], reduced_data[i, 1], label=sub.get('diagnostic'))
# plt.legend()
# plt.show()

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(12,6))
for ax, s, subID in zip(axs.flat, atrophy_thickness, atrophy_ids):
    viz.plot_spectrum(consensus.G, s, ax, no_dc=True)
    ax.set_title(id2diag.get(subID))
    
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,5))
axs[0].plot(np.stack(healthy_thickness).T, alpha=.3)
axs[0].set_title('Healthy thicknesses')
axs[1].plot(np.stack(atrophy_thickness).T, alpha=.6)
axs[1].set_title('Atrophied thicknesses')
plt.show()

In [ ]:
subset=[1,5]
fig, axs = plt.subplots(1,2, figsize=(12,5))
axs[0].plot(np.stack(gft_controls).T[1:20,:], alpha=.3)
#axs[0].plot(np.stack(gft_controls).T, alpha=.3)

axs[0].set_title('Healthy thicknesses GFTs')
axs[0].set_xticks(np.arange(19))
axs[0].set_xticklabels(np.arange(19)+1)
axs[0].grid(axis='x')
axs[1].plot(np.stack(gft_atrophies).T[1:20,subset]-np.stack([avg_gft[1:20] for _ in range(len(subset))]).T, alpha=.8)
#axs[1].plot(np.stack(gft_atrophies).T[1:40,:], alpha=.8)

axs[1].set_title('Atrophied thicknesses GFTs')
#axs[1].legend([id2diag.get(id) for id in atrophy_ids])
axs[1].set_xticks(np.arange(19))
axs[1].set_xticklabels(np.arange(19)+1)
#axs[1].set_ylim([-2.4,2.4])
axs[1].grid(axis='x')
plt.show()

In [ ]:
# compute mean gft of thickness for controls
mean_gft = gft_controls.mean(axis=0)
std_gft = gft_controls.std(axis=0)

In [ ]:
ids_diff = []
for s in gft_atrophies:
    diff = np.abs(s-mean_gft)
    diff = diff[1:]
    ids_diff.append(diff.argsort()[-10:][::-1]+1)
    

In [ ]:
ids_diff

In [ ]:
atrophy_ids

In [ ]:
i=-3
viz.plot_signal_3d(consensus, atrophy_thickness[i]-avg_healthy_thick, mesh_type='pial_whole', cmap="Spectral", black_bg=False, title=str(atrophy_ids[i]))

In [ ]:
i = 3
viz.plot_signal_3d(consensus, i, mesh_type='pial_whole', cmap="Spectral", black_bg=False, title='eigenmode {}'.format(i))

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(12,6))
for ax, s_hat, subID in zip(axs.flat, gft_atrophies, atrophy_ids):
    ax.stem(x, np.abs(s_hat-mean_gft)[1:], markerfmt='')
    ax.set_title(subID)
    
fig.tight_layout()
plt.show()

# Smoothness

In [ ]:
def compute_smoothness(signal):
    s = signal/np.amax(signal)
    return s.T@consensus.G.L@s

In [ ]:
thick_healthy = []
for t in healthy_thickness:
    thick_smoothness = compute_smoothness(t)
    thick_healthy.append(thick_smoothness)

In [ ]:
thick_atrophy = []
for t in atrophy_thickness:
    thick_smoothness = compute_smoothness(t)
    thick_atrophy.append(thick_smoothness)

In [ ]:
# Create a violin plot for the first list
# Create a swarm plot for the second list
fig, ax = plt.subplots()

sns.violinplot(y=thick_healthy)
sns.swarmplot(y=thick_atrophy, color='red', size=10)
ax.set_ylim(1.5,2.8)

# Optionally, set the title and labels for the axes
plt.ylabel('Thickness')

# Show the plot
plt.show()